In [1]:
import sys
sys.path.append('home/adam/_modules')
from cqjupyter import *

In [2]:
#####
# Inputs
######
lbumps = 1       # number of bumps long
wbumps = 1        # number of bumps wide
thin = False  # True for thin, False for thick

#
# Lego Brick Constants-- these make a lego brick a lego :)
#

# add a c method to 'capture' constants so they don't get collected by CQGI as alterable
def c():
    pass

c.pitch = 8.0
c.clearance = 0.1
c.bumpDiam = 4.8
c.bumpHeight = 1.8
if thin:
    height = 3.2
else:
    height = 9.6

t = (c.pitch - (2 * c.clearance) - c.bumpDiam) / 2.0
postDiam = c.pitch - t  # works out to 6.5
total_length = lbumps*c.pitch - 2.0*c.clearance
total_width = wbumps*c.pitch - 2.0*c.clearance


# make the base
s = cq.Workplane("XY").box(total_length, total_width, height)

# shell inwards not outwards
s = s.faces("<Z").shell(-1.0 * t)

# make the bumps on the top
s = s.faces(">Z").workplane(). \
    rarray(c.pitch, c.pitch, int(lbumps), int(wbumps), True).circle(c.bumpDiam / 2.0) \
    .extrude(c.bumpHeight)

# add posts on the bottom. posts are different diameter depending on geometry
# solid studs for 1 bump, tubes for multiple, none for 1x1
tmp = s.faces("<Z").workplane(invert=True)

if lbumps > 1 and wbumps > 1:
    tmp = tmp.rarray(c.pitch, c.pitch, int(lbumps - 1), int(wbumps - 1), center=True). \
        circle(postDiam / 2.0).circle(c.bumpDiam / 2.0).extrude(height - t)
elif lbumps > 1:
    tmp = tmp.rarray(c.pitch, c.pitch, int(lbumps - 1), 1, center=True). \
        circle(t).extrude(height - t)
elif wbumps > 1:
    tmp = tmp.rarray(c.pitch, c.pitch, 1, int(wbumps - 1), center=True). \
        circle(t).extrude(height - t)
else:
    tmp = s
    
build_object(tmp)